In [1]:
from google.colab import files
uploaded = files.upload()


Saving empleadosRETO.csv to empleadosRETO.csv


In [15]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Paso 1: Importar las librerías requeridas.

# Paso 2: Leer el archivo CSV llamado empleadosRETO.csv y colocar los datos en un DataFrame de Pandas llamado EmpleadosAttrition.
EmpleadosAttrition = pd.read_csv("empleadosRETO.csv")

# Paso 3: Eliminar las columnas que no tienen relación con la salida.
columns_to_drop = ["EmployeeCount", "EmployeeNumber", "Over18", "StandardHours"]
EmpleadosAttrition = EmpleadosAttrition.drop(columns=columns_to_drop)

# Paso 4: Crear una columna llamada Year y obtener el año de contratación del empleado a partir de su fecha 'HiringDate'.
def get_hiring_year(date_string):
    try:
        return pd.to_datetime(date_string, format='%m/%d/%Y').year
    except ValueError:
        return pd.NaT

EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].apply(get_hiring_year)

# Paso 5: Crear una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018.
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']

# Paso 6: Renombrae la variable DistanceFromHome a DistanceFromHome_km y convertir los valores a enteros.
EmpleadosAttrition.rename(columns={"DistanceFromHome": "DistanceFromHome_km"}, inplace=True)
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.extract('(\d+)').astype(int)

# Paso 7: Borrar las columnas Year, HiringDate y DistanceFromHome_km.
EmpleadosAttrition = EmpleadosAttrition.drop(columns=["Year", "HiringDate", "DistanceFromHome_km"])

# Paso 8: Escalar la variable MonthlyIncome para que tenga un valor entre 0 y 1.
scaler = StandardScaler()
EmpleadosAttrition['MonthlyIncome'] = scaler.fit_transform(EmpleadosAttrition['MonthlyIncome'].values.reshape(-1, 1))

# Paso 9: Convertir las variables categóricas restantes a numéricas.
EmpleadosAttrition = pd.get_dummies(EmpleadosAttrition, columns=["BusinessTravel", "Department", "EducationField", "Gender", "JobRole", "MaritalStatus", "Attrition"], drop_first=True)

# Paso 10: Calcular la correlación lineal de cada variable con respecto a 'Attrition_Yes'.
correlations = EmpleadosAttrition.corr()['Attrition_Yes']

# Paso 11: Seleccionar solo las variables con una correlación mayor o igual a 0.1.
selected_columns = correlations[correlations >= 0.1].index
EmpleadosAttritionFinal = EmpleadosAttrition[selected_columns].copy()

# Paso 12: Realizar el análisis de componentes principales (PCA).
n_components = 4  # El número de componentes principales que se desea mantener
pca = PCA(n_components=n_components)
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal.drop(columns="Attrition_Yes"))

# Paso 13: Agregar el mínimo número de Componentes Principales en columnas del DataFrame.
for i in range(n_components):
    column_name = f'PC{i}'
    EmpleadosAttritionFinal[column_name] = EmpleadosAttritionPCA[:, i].copy()

# Paso 14: Guardar el set de datos en un archivo CSV y descárgalo.
EmpleadosAttritionFinal.to_csv("EmpleadosAttritionFinal.csv", index=False)

# Paso 15: Descargar el archivo CSV
from google.colab import files
files.download("EmpleadosAttritionFinal.csv")


<ipython-input-15-785cfd13d1fe>:42: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = EmpleadosAttrition.corr()['Attrition_Yes']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>